In [ ]:
%%time
import xarray as xr
import fsspec
import pandas as pd
import hvplot.xarray
import numpy as np

In [ ]:
fs = fsspec.filesystem('file')

In [ ]:
zlist = sorted(fs.glob('/caldera/projects/usgs/water/wbeep/conus404_work/test1/target_0*'))

In [ ]:
zlist[-1]

In [ ]:
zarr_whole = '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/conus404/conus404_whole.zarr'

In [ ]:
fs.ls('/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/conus404/')

In [ ]:
ds0 = xr.open_dataset(zlist[0], engine='zarr', chunks={})
ds0.time[0].values

In [ ]:
ds1 = xr.open_dataset(zlist[-1], engine='zarr', chunks={})
ds1.time[-1].values

In [ ]:
dates = pd.date_range(start=ds0.time[0].values, end=ds1.time[-1].values, freq='1h')

In [ ]:
drop_vars = ['DZS', 'HGT', 'ISLTYP', 'IVGTYP', 'LAKEMASK', 'LANDMASK', 'LU_INDEX', 'MUB', 'P00', 'PB', 'PHB', 'P_TOP', 'SHDMAX', 'SHDMIN', 'SNOALB', 'T00', 'TISO', 'TLP', 'TSK_FORCE', 'TSK_FORCE_TEND', 'VAR', 'VAR_SSO', 'XLAND', 'ZETATOP', 'ZS', 'lat', 'lat_u', 'lat_v', 'lon', 'lon_u', 'lon_v']

In [ ]:
source_dataset = ds0.drop(drop_vars)

In [ ]:
template = (
      source_dataset
      .chunk()
      .pipe(xr.zeros_like)
      .isel(time=0, drop=True)
      .expand_dims(time=len(dates))
  )

In [ ]:
template['time'] = dates

In [ ]:
%%time
tchunk = 144
template = template.chunk({'time':tchunk})

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client()

In [ ]:
client

In [ ]:
%%time
template.to_zarr(zarr_whole, compute=False, consolidated=True, mode='w')   #writes no data

In [ ]:
%%time
ds0.drop(drop_vars).to_zarr(zarr_whole, region = {'time': slice(0, 144)})  #writes

In [ ]:
template.ACECAN

In [ ]:
ds0[drop_vars].to_zarr(zarr_whole, mode='a')

In [ ]:
%%time

for i in range(1,4):
    start = i*tchunk
    stop =(i+1)*tchunk
    print(zlist[i])
    dsi = xr.open_dataset(zlist[i], engine='zarr', chunks={})
    dsi.to_zarr(zarr_whole, region = {'time': slice(start, stop)})

In [ ]:
ds = xr.open_dataset(zarr_whole, engine='zarr', consolidated=True)

In [ ]:
ds.U10[:1000,500,500].hvplot(x='time', grid=True)

In [ ]:
ds.U10[0,:,:].hvplot.quadmesh(x='lon', y='lat', geo=True, tiles='OSM',
                              cmap='turbo', rasterize=True)